In [1]:
from sys import platform
import numpy as np
import subprocess
import cv2 as cv  # OpenCV Library
import shutil
import ast
import sys
import os
from torchvision.datasets import ImageFolder
import torchvision
from torchvision.models import ResNet18_Weights
from torchvision import datasets, models, transforms
from torch.utils.data.dataloader import DataLoader
from torch.autograd import Variable
from torch.utils.data import random_split
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.nn as nn
import torch
from PIL import Image

transform = transforms.Compose(
    [
        transforms.Resize((50, 50)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# Path of generated arena image
arena_path = "/mnt/Storage Drive/Projects/E-YRC/EYRC_2023/Task 2/Task 2C/arena.png"
event_list = []
detected_list = []
combat = "combat"
rehab = "humanitarianaid"
military_vehicles = "militaryvehicles"
fire = "fire"
destroyed_building = "destroyedbuilding"

In [2]:
# NOTE: This function has already been done for you, don't make any changes in it.
def arena_image(arena_path):
    """
        Purpose:
        ---
        This function will take the path of the generated image as input and
    read the image specified by the path.

        Input Arguments:
        ---
        `arena_path`: Generated image path i.e. arena_path (declared above)

        Returns:
        ---
        `arena` : [ Numpy Array ]

        Example call:
        ---
        arena = arena_image(arena_path)
    """
    """
    ADD YOUR CODE HERE
    """
    frame = cv.imread(arena_path)
    arena = cv.resize(frame, (700, 700))
    # cv.imshow("a", frame)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    return arena

In [3]:
arena_image(arena_path)
print()

In [10]:
# NOTE: You can tweak this function in case you need to give more inputs
def event_identification(arena):
    """
        Purpose:
        ---
        This function will select the events on arena image and extract them as
    separate images.

        Input Arguments:
        ---
        `arena`: Image of arena detected by arena_image()

        Returns:
        ---
        `event_list`,  : [ List ]
                            event_list will store the extracted event images

        Example call:
        ---
        event_list = event_identification(arena)
    """
    """
    ADD YOUR CODE HERE
    """
    arena_org = arena.copy()
    image_cnts = 0
    border_width = 5
    arena_gray = cv.cvtColor(arena, cv.COLOR_BGR2GRAY)
    arena_gray = cv.GaussianBlur(arena_gray, (7, 7), 1)
    ret, thresh = cv.threshold(arena_gray, 250, 255, cv.THRESH_BINARY)
    contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    print("Number of contours detected:", len(contours))
    for cnt in contours:
        approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
        (x, y) = cnt[0, 0]
        if len(approx) == 4:
            x1, y1, w, h = cv.boundingRect(cnt)
            ratio = float(w) / h
            area = w * h
            if ratio >= 0.9 and ratio <= 1.1 and area > 20:
                arena = cv.drawContours(arena, [cnt], -1, (0, 255, 255), 3)
                cv.putText(
                    arena,
                    "Square",
                    (x, y),
                    cv.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    (255, 255, 0),
                    2,
                )
                print(x1, " ", y1)
                cropped_image = arena_org[
                    y1 + border_width : y1 + 50 - border_width,
                    x1 + border_width : x1 + 50 - border_width,
                ]
                event_list.append([cropped_image])

                cv.imwrite(
                    "imgdir/cropped_image_" + str(image_cnts) + ".jpg", cropped_image
                )

                image_cnts += 1

    cv.imshow("Event Identification", arena)
    cv.imshow("Thresh", thresh)
    cv.waitKey(0)
    cv.destroyAllWindows()
    print("len", len(event_list[0]))
    # print(event_list[0])
    return event_list


event_list= event_identification(arena_image(arena_path))
cv.imshow("el", event_list[0][0])
cv.waitKey(0)
cv.destroyAllWindows()

Number of contours detected: 33
151   595
456   464
461   332
142   331
154   117
len 1


In [5]:
def predict(image):
    class_names = [
        "combat",
        "destroyedbuilding",
        "fire",
        "humanitarianaid",
        "militaryvehicles",
    ]
    image = Image.open(image)
    model = torch.load("model_conv_2c.pth")
    model.eval()
    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    output = model(image_tensor)
    index = output.data.numpy().argmax()
    event = class_names[index]
    print(event)
    return event


for i in range(0, 5):
    print(i, ": ", end="")
    image = "imgdir/cropped_image_" + str(i) + ".jpg"
    predict(image)

0 : fire
1 : militaryvehicles
2 : militaryvehicles
3 : combat
4 : humanitarianaid


In [6]:
def classify_event(image):
    """
        Purpose:
        ---
        This function will load your trained model and classify the event from an image which is
    sent as an input.

        Input Arguments:
        ---
        `image`: Image path sent by input file

        Returns:
        ---
        `event` : [ String ]
                                                  Detected event is returned in the form of a string

        Example call:
        ---
        event = classify_event(image_path)
    """
    """
    ADD YOUR CODE HERE
    """
    event = "variable to return the detected function"
    class_names = [
        "combat",
        "destroyedbuilding",
        "fire",
        "humanitarianaid",
        "militaryvehicles",
    ]
    image = Image.open(image)
    model = torch.load("model_conv.pth")
    model.eval()
    transform = transforms.Compose(
        [
            transforms.Resize((50, 50)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    )
    image_tensor = transform(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    output = model(image_tensor)
    index = output.data.numpy().argmax()
    event = class_names[index]
    return event

In [7]:
########################### DO NOT MAKE ANY CHANGES IN THE SCRIPT BELOW ###########################
def classification(event_list):
    for img_index in range(0, 5):
        img = event_list[img_index]
        detected_event = classify_event(img)
        print((img_index + 1), detected_event)
        if detected_event == combat:
            detected_list.append("combat")
        if detected_event == rehab:
            detected_list.append("rehab")
        if detected_event == military_vehicles:
            detected_list.append("militaryvehicles")
        if detected_event == fire:
            detected_list.append("fire")
        if detected_event == destroyed_building:
            detected_list.append("destroyedbuilding")
    os.remove("arena.png")
    return detected_list


def detected_list_processing(detected_list):
    try:
        detected_events = open("detected_events.txt", "w")
        detected_events.writelines(str(detected_list))
    except Exception as e:
        print("Error: ", e)


def input_function():
    if platform == "win32":
        try:
            subprocess.run("input.exe")
        except Exception as e:
            print("Error: ", e)
    if platform == "linux":
        try:
            subprocess.run("./input")
        except Exception as e:
            print("Error: ", e)


def output_function():
    if platform == "win32":
        try:
            subprocess.run("output.exe")
        except Exception as e:
            print("Error: ", e)
    if platform == "linux":
        try:
            subprocess.run("./output")
        except Exception as e:
            print("Error: ", e)


###################################################################################################
def main():
    ##### Input #####
    input_function()
    #################

    ##### Process #####
    arena = arena_image(arena_path)
    event_list = event_identification(arena)
    detected_list = classification(event_list)
    detected_list_processing(detected_list)
    ###################

    ##### Output #####
    output_function()
    ##################


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("Interrupted")
        if os.path.exists("arena.png"):
            os.remove("arena.png")
        if os.path.exists("detected_events.txt"):
            os.remove("detected_events.txt")
        sys.exit()

Error:  [Errno 2] No such file or directory: './input'


Number of contours detected: 33
151   595
456   464
461   332
142   331
154   117
len 1


AttributeError: 'list' object has no attribute 'read'